## PromoMix - Gurobi Optimization Code

In [78]:
import numpy as np
import pandas as pd
import math

In [79]:
import gurobipy as gp
from gurobipy import GRB

In [157]:
promomix_df = pd.read_excel('C:/Users/ANANDA28/Desktop/Work/Promo Mix Simulator/Gurobi Exploration/Input Data - PromoMix.xlsx',sheet_name = "Raw Data")
response_curve_df = pd.read_excel('C:/Users/ANANDA28/Desktop/Work/Promo Mix Simulator/Gurobi Exploration/Input Data - PromoMix.xlsx', sheet_name='Response Curve')

In [158]:
print(promomix_df.shape)
print(promomix_df.columns.tolist())
print('\n')
print(response_curve_df.shape)
print(response_curve_df.columns.tolist())

(41, 125)
['country', 'brand', 'segment', 'period', 'level', 'segment_level', 'channel_category', 'channel_type', 'channel', 'sub_channel', 'segment_description', 'last_refresh_date', 'time_id', 'lt_roi_lower_metric_value_max', 'sales_per_activity', 'lt_factor', 'active_months', 'gross_lt_incremental_revenue_per_activity', 'realized_contribution_upper', 'realized_incremental_rx_upper', 'lt_contribution_upper', 'lt_contribution_lower', 'spend_per_activity', 'ad_stock_factor', 'lt_incremental_rx', 'c_mroi_metric_value_max', 'lt_incremental_rx_per_activity', 'lt_incremental_revenue_lower', 'lt_incremental_revenue_upper', 'incremental_rx_year1', 'gross_realized_roi', 'brand_trx', 'no_of_bricks', 'incremental_revenue_from_model_upper', 'major_active_months', 'incremental_revenue_from_model', 'incremental_rx_per_activity', 'gross_realized_incremental_revenue_per_activity', 'net_realized_roi', 'realized_incremental_revenue_lower', 'secondary_spend_per_activity', 'realized_roi_lower_metric_val

In [330]:
# Initialize with default bounds
margin = 20

# user inputs from frontend

promomix_df['spend_lower_bound'] = (1-(margin/100))*promomix_df['spend']
promomix_df['spend_upper_bound'] = (1+(margin/100))*promomix_df['spend']

promomix_df['opt_max_func'] = "Profit" # Two options : 1.Sales ; 2.Profit
promomix_df['opt_max_criteria'] = "Change spend to" # Three options : 1.Recommend an Optimal Spend ; 2.Only reallocate Spend ; 3.Change spend to
promomix_df['opt_channel_level'] = "National"  # Two options for level : 1.National ; 2.Segment 
promomix_df['user_inp_change_spend_to'] =  300000000



In [331]:
inp_df = promomix_df
inp_resp_df = response_curve_df

In [332]:
########################################################################
## 1. Pre - Processing on Input table based on constraints selections ##
########################################################################

# Modify Input Data based on National/ Segment Selection
chnl_natl_only = [x for x in inp_df[inp_df['level'].str.lower() == 'level 4 - subchannel']['channel'].unique() if x not in inp_df[inp_df['level'].str.lower() == 'segment']['channel'].unique()]

if(''.join(inp_df['opt_channel_level'].unique()).lower() == 'national'):
    inp_df = inp_df[inp_df['level'].str.lower() == 'level 4 - subchannel']
elif(''.join(inp_df['opt_channel_level'].unique()).lower() == 'segment'):
    inp_df = pd.merge(inp_df,inp_df[(np.logical_not(inp_df['channel'].isin(chnl_natl_only))) & (inp_df['level'].str.lower() == 'level 4 - subchannel')][['channel','spend_lower_bound','spend_upper_bound']].rename(columns={'spend_lower_bound':'natl_spend_lower_bound','spend_upper_bound':'natl_spend_upper_bound'}),on=['channel',],how='left')
    inp_df = inp_df[(inp_df['level'].str.lower() == 'segment') | ((inp_df['level'].str.lower() == 'level 4 - subchannel') & (inp_df['channel'].isin(chnl_natl_only)))]

# Identify channel - segment selected by user for optimization
channels_to_opt = inp_df[inp_df['opt_chnl_selected_flg']==1][['channel', 'segment']].agg(tuple, 1).unique()

# Sales/Profit Maximization Criteria (Remove the component of spends which are not to be optimized)

if(''.join(inp_df['opt_max_criteria'].unique()).lower() =='only reallocate spend'):
    total_spend = inp_df['spend'].sum() - (inp_df[inp_df['opt_chnl_selected_flg']==0]['spend'].sum() if 0 in inp_df['opt_chnl_selected_flg'] else 0)
elif(''.join(inp_df['opt_max_criteria'].unique()).lower() =='change spend to'):
    total_spend = float(inp_df['user_inp_change_spend_to'].unique()) - (inp_df[inp_df['opt_chnl_selected_flg']==0]['spend'].sum() if 0 in inp_df['opt_chnl_selected_flg'] else 0 )
# elif(''.join(inp_df['opt_max_criteria'].unique()).lower() =='recommend optimal spend'):
#     total_spend = 100000*inp_df['spend'].sum() - (inp_df[inp_df['opt_chnl_selected_flg']==0]['spend'].sum() if 0 in inp_df['opt_chnl_selected_flg'] else 0 ) 


In [249]:
print(total_spend)
print(channels_to_opt)

299703565.0
[('DTC DISPLAY-RA PSA BRANDED', 'NATIONAL')
 ('DTC DISPLAY-UC BRANDED', 'NATIONAL')
 ('DTC PRINT-RA BRANDED', 'NATIONAL')
 ('DTC SEARCH-RA PSA AGNOSTIC BRANDED', 'NATIONAL')
 ('DTC SEARCH-UC BRANDED', 'NATIONAL')
 ('DTC SOCIAL-RA PSA BRANDED', 'NATIONAL')
 ('DTC SOCIAL-UC BRANDED', 'NATIONAL') ('DTC TV-PSA BRANDED', 'NATIONAL')
 ('DTC TV-RA BRANDED', 'NATIONAL') ('DTC TV-RHEUM BRANDED', 'NATIONAL')
 ('DTC TV-UC BRANDED', 'NATIONAL')
 ('DTC VIDEO-RA PSA RHEUM BRANDED', 'NATIONAL')
 ('DTC VIDEO-UC BRANDED', 'NATIONAL') ('PFIZER DETAILS', 'NATIONAL')
 ('PFIZER SAMPLES', 'NATIONAL') ('SPEAKER', 'NATIONAL')]


In [164]:
for i in range(len(channels_to_opt)):
    print(channels_to_opt[i][0])

DTC DISPLAY-RA PSA BRANDED
DTC DISPLAY-UC BRANDED
DTC PRINT-RA BRANDED
DTC SEARCH-RA PSA AGNOSTIC BRANDED
DTC SEARCH-UC BRANDED
DTC SOCIAL-RA PSA BRANDED
DTC SOCIAL-UC BRANDED
DTC TV-PSA BRANDED
DTC TV-RA BRANDED
DTC TV-RHEUM BRANDED
DTC TV-UC BRANDED
DTC VIDEO-RA PSA RHEUM BRANDED
DTC VIDEO-UC BRANDED
PFIZER DETAILS
PFIZER SAMPLES
SPEAKER


In [333]:
opt_tuple = []
num_channels = len(channels_to_opt)

In [334]:
env.dispose()

In [335]:
env.close()

In [336]:
env = gp.Env(empty=True)
env.setParam("CSManager","10.192.71.173:61080")
env.setParam("CSAPIAccessID", "5bcf91fe-dbd5-4b16-bb1e-676001613076")
env.setParam("CSAPISecret", "ad261c20-395a-4a86-b059-bd6d7779d132")
env.start()

Changed value of parameter CSManager to 10.192.71.173:61080
   Prev:   Default: 
Changed value of parameter CSAPIAccessID to 5bcf91fe-dbd5-4b16-bb1e-676001613076
   Prev:   Default: 
Changed value of parameter CSAPISecret
Using license file C:\gurobi903\gurobi.lic
Set parameter CSManager to value 10.192.71.173:61080
Set parameter CSAPIAccessID to value 5bcf91fe-dbd5-4b16-bb1e-676001613076
Set parameter CSAPISecret
Compute Server job ID: d881a3b7-d24c-482f-8f9a-d3deb044fc7e
Capacity available on '10.192.71.215' - connecting...
Established HTTP unencrypted connection


<gurobipy.Env, Parameter changes: JobID=d881a3b7-d24c-482f-8f9a-d3deb044fc7e, CSManager=10.192.71.173:61080, CSAPIAccessID=5bcf91fe-dbd5-4b16-bb1e-676001613076, CSAPISecret=(user-defined)>

In [337]:
model = gp.Model("promomix-model", env=env)

In [338]:
x = {}
y = {}

for i in range(num_channels):
    x[i] = model.addVar(vtype = GRB.CONTINUOUS,lb = float(inp_df[inp_df[['channel', 'segment']].agg(tuple, 1) == channels_to_opt[i]]['spend_lower_bound']),ub = float(inp_df[inp_df[['channel', 'segment']].agg(tuple, 1) == channels_to_opt[i]]['spend_upper_bound']), name = ('|'.join(channels_to_opt[i]) + '|' + 'Spend'))
    y[i] = model.addVar(vtype = GRB.CONTINUOUS, lb = (-gp.GRB.INFINITY), ub = gp.GRB.INFINITY, name = ('|'.join(channels_to_opt[i]) + '|' + 'Revenue'))

# Constraint:
if(''.join(inp_df['opt_max_criteria'].unique()).lower()!='recommend an optimal spend'):
    model.addConstr(gp.quicksum(x[i] for i in range(num_channels)) == total_spend, "Total Spend Constraint ")

# Add piecewise-linear constraint - Response Curves

if(''.join(inp_df['opt_max_func'].unique()).lower() == "sales"):
    for i in range(num_channels):
        model.addGenConstrPWL(x[i], y[i], inp_resp_df[inp_resp_df[['channel', 'segment']].agg(tuple, 1) == channels_to_opt[i]]['spend'].tolist(), inp_resp_df[inp_resp_df[['channel', 'segment']].agg(tuple, 1) == channels_to_opt[i]]['net_lt_incremental_revenue'].tolist(), "pwl")
elif(''.join(inp_df['opt_max_func'].unique()).lower() == "profit"):
     for i in range(num_channels):
        model.addGenConstrPWL(x[i], y[i], inp_resp_df[inp_resp_df[['channel', 'segment']].agg(tuple, 1) == channels_to_opt[i]]['spend'].tolist(), inp_resp_df[inp_resp_df[['channel', 'segment']].agg(tuple, 1) == channels_to_opt[i]]['net_profit'].tolist(), "pwl")


In [ ]:
mycts = m.addConstrs((quicksum(x[i] for i in range() for p in P[i,f]) <= z[f,t]), "myconstraint")

In [217]:
model.update()

In [218]:
model.getVars()

[<gurobi.Var DTC DISPLAY-RA PSA BRANDED|NATIONAL|Spend>,
 <gurobi.Var DTC DISPLAY-RA PSA BRANDED|NATIONAL|Revenue>,
 <gurobi.Var DTC DISPLAY-UC BRANDED|NATIONAL|Spend>,
 <gurobi.Var DTC DISPLAY-UC BRANDED|NATIONAL|Revenue>,
 <gurobi.Var DTC PRINT-RA BRANDED|NATIONAL|Spend>,
 <gurobi.Var DTC PRINT-RA BRANDED|NATIONAL|Revenue>,
 <gurobi.Var DTC SEARCH-RA PSA AGNOSTIC BRANDED|NATIONAL|Spend>,
 <gurobi.Var DTC SEARCH-RA PSA AGNOSTIC BRANDED|NATIONAL|Revenue>,
 <gurobi.Var DTC SEARCH-UC BRANDED|NATIONAL|Spend>,
 <gurobi.Var DTC SEARCH-UC BRANDED|NATIONAL|Revenue>,
 <gurobi.Var DTC SOCIAL-RA PSA BRANDED|NATIONAL|Spend>,
 <gurobi.Var DTC SOCIAL-RA PSA BRANDED|NATIONAL|Revenue>,
 <gurobi.Var DTC SOCIAL-UC BRANDED|NATIONAL|Spend>,
 <gurobi.Var DTC SOCIAL-UC BRANDED|NATIONAL|Revenue>,
 <gurobi.Var DTC TV-PSA BRANDED|NATIONAL|Spend>,
 <gurobi.Var DTC TV-PSA BRANDED|NATIONAL|Revenue>,
 <gurobi.Var DTC TV-RA BRANDED|NATIONAL|Spend>,
 <gurobi.Var DTC TV-RA BRANDED|NATIONAL|Revenue>,
 <gurobi.Var DTC

In [339]:
# Objective Function
model.setObjective(gp.quicksum(y[i] for i in range(num_channels)), GRB.MAXIMIZE)

In [340]:
model.optimize()

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 1 rows, 32 columns and 16 nonzeros
Model fingerprint: 0x7b691f70
Model has 16 general constraints
Variable types: 32 continuous, 0 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+06, 9e+07]
  RHS range        [3e+08, 3e+08]
Presolve added 32 rows and 640 columns
Presolve time: 0.00s
Presolved: 33 rows, 672 columns, 1344 nonzeros
Presolved model has 16 SOS constraint(s)
Variable types: 672 continuous, 0 integer (0 binary)

Root relaxation: objective 1.676211e+08, 77 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    1.676211e+08 1.6762e+08  0.00%     -    0s

Explored 0 nodes (77 simplex iterations) in 0.01 seconds
Thread count was 16 (of 16 available processors)

Solutio

In [341]:
opt_tuple = []

In [342]:
for v in model.getVars():
    if "spend" in v.Varname.lower():
        opt_tuple = opt_tuple + [(v.Varname.split('|')[0].strip(),v.Varname.split('|')[1].strip(), v.X),]


In [343]:
env.dispose()

In [344]:
opt_tuple

[('DTC DISPLAY-RA PSA BRANDED', 'NATIONAL', 12353360.652),
 ('DTC DISPLAY-UC BRANDED', 'NATIONAL', 5875494.516),
 ('DTC PRINT-RA BRANDED', 'NATIONAL', 4515165.612),
 ('DTC SEARCH-RA PSA AGNOSTIC BRANDED', 'NATIONAL', 22393576.74),
 ('DTC SEARCH-UC BRANDED', 'NATIONAL', 5011156.152),
 ('DTC SOCIAL-RA PSA BRANDED', 'NATIONAL', 7707731.052),
 ('DTC SOCIAL-UC BRANDED', 'NATIONAL', 10279567.464),
 ('DTC TV-PSA BRANDED', 'NATIONAL', 3897042.992),
 ('DTC TV-RA BRANDED', 'NATIONAL', 28907725.672000006),
 ('DTC TV-RHEUM BRANDED', 'NATIONAL', 34887806.364),
 ('DTC TV-UC BRANDED', 'NATIONAL', 41934577.153600045),
 ('DTC VIDEO-RA PSA RHEUM BRANDED', 'NATIONAL', 15554437.488),
 ('DTC VIDEO-UC BRANDED', 'NATIONAL', 11201992.7424),
 ('PFIZER DETAILS', 'NATIONAL', 88708497.6),
 ('PFIZER SAMPLES', 'NATIONAL', 2200123.2),
 ('SPEAKER', 'NATIONAL', 4275309.6)]

In [19]:
# Join the optimized spend with input data
opt_df = inp_df.merge(pd.DataFrame(opt_tuple, columns=['channel', 'segment', 'sim_spend']),how='left', left_on=['channel','segment'], right_on = ['channel','segment'])
opt_df['sim_spend'] = opt_df['sim_spend'].fillna(opt_df['spend'])

In [21]:
print(opt_df['sim_spend'].sum())
print(opt_df['spend'].sum())

279492130.53000003
279492130.53


In [24]:
print(opt_df['sim_spend'][0:10])
print(opt_df['spend'][0:10])

0    1.321250e+05
1    1.235336e+07
2    5.875495e+06
3    4.515166e+06
4    1.753866e+07
5    3.841886e+06
6    7.707731e+06
7    7.881002e+06
8    3.897043e+06
9    2.890773e+07
Name: sim_spend, dtype: float64
0      132125.00
1    10294467.21
2     4896245.43
3     3762638.01
4    18661313.95
5     4175963.46
6     6423109.21
7     8566306.22
8     4871303.74
9    36134657.09
Name: spend, dtype: float64


### Batch Optimization

In [57]:
env = gp.Env(empty=True)
env.setParam('LogFile','batchmode.log')
env.setParam("CSManager","10.192.71.173:61080")
env.setParam("CSAPIAccessID", "5bcf91fe-dbd5-4b16-bb1e-676001613076")
env.setParam("CSAPISecret", "ad261c20-395a-4a86-b059-bd6d7779d132")
env.setParam('CSBatchMode',1)
env.start()

Changed value of parameter LogFile to batchmode.log
   Prev:   Default: 
Changed value of parameter CSManager to 10.192.71.173:61080
   Prev:   Default: 
Changed value of parameter CSAPIAccessID to 5bcf91fe-dbd5-4b16-bb1e-676001613076
   Prev:   Default: 
Changed value of parameter CSAPISecret
Changed value of parameter CSBatchMode to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Using license file C:\gurobi903\gurobi.lic
Set parameter CSManager to value 10.192.71.173:61080
Set parameter CSAPIAccessID to value 5bcf91fe-dbd5-4b16-bb1e-676001613076
Set parameter CSAPISecret
Set parameter CSBatchMode to value 1
Set parameter LogFile to value batchmode.log


<gurobipy.Env, Parameter changes: CSManager=10.192.71.173:61080, CSAPIAccessID=5bcf91fe-dbd5-4b16-bb1e-676001613076, CSAPISecret=(user-defined), CSBatchMode=1, LogFile=batchmode.log>

In [58]:
model = gp.Model("promomix-model", env=env)

In [59]:
x = {}
y = {}

for i in range(num_channels):
    x[i] = model.addVar(vtype = GRB.CONTINUOUS,lb = float(inp_df[inp_df[['channel', 'segment']].agg(tuple, 1) == channels_to_opt[i]]['spend_lower_bound']),ub = float(inp_df[inp_df[['channel', 'segment']].agg(tuple, 1) == channels_to_opt[i]]['spend_upper_bound']), name = ('|'.join(channels_to_opt[i]) + '|' + 'Spend'))
    y[i] = model.addVar(vtype = GRB.CONTINUOUS, lb = (-gp.GRB.INFINITY), ub = gp.GRB.INFINITY, name = ('|'.join(channels_to_opt[i]) + '|' + 'Revenue'))

# Constraint:
if(''.join(inp_df['opt_max_criteria'].unique()).lower()!='recommend optimal spend'):
    model.addConstr(gp.quicksum(x[i] for i in range(num_channels)) == total_spend, "Total Spend Constraint ")

# Add piecewise-linear constraint - Response Curves

if(''.join(inp_df['opt_max_func'].unique()).lower() == "sales"):
    for i in range(num_channels):
        model.addGenConstrPWL(x[i], y[i], inp_resp_df[inp_resp_df[['channel', 'segment']].agg(tuple, 1) == channels_to_opt[i]]['spend'].tolist(), inp_resp_df[inp_resp_df[['channel', 'segment']].agg(tuple, 1) == channels_to_opt[i]]['net_lt_incremental_revenue'].tolist(), "pwl")
elif(''.join(inp_df['opt_max_func'].unique()).lower() == "profit"):
     for i in range(num_channels):
        model.addGenConstrPWL(x[i], y[i], inp_resp_df[inp_resp_df[['channel', 'segment']].agg(tuple, 1) == channels_to_opt[i]]['spend'].tolist(), inp_resp_df[inp_resp_df[['channel', 'segment']].agg(tuple, 1) == channels_to_opt[i]]['net_lt_profit'].tolist(), "pwl")

# Objective Function

model.setObjective(gp.quicksum(y[i] for i in range(num_channels)), GRB.MAXIMIZE)


In [69]:
model.getVars()

[<gurobi.Var DTC DISPLAY-RA PSA BRANDED|NATIONAL|Spend>,
 <gurobi.Var DTC DISPLAY-RA PSA BRANDED|NATIONAL|Revenue>,
 <gurobi.Var DTC DISPLAY-UC BRANDED|NATIONAL|Spend>,
 <gurobi.Var DTC DISPLAY-UC BRANDED|NATIONAL|Revenue>,
 <gurobi.Var DTC PRINT-RA BRANDED|NATIONAL|Spend>,
 <gurobi.Var DTC PRINT-RA BRANDED|NATIONAL|Revenue>,
 <gurobi.Var DTC SEARCH-RA PSA AGNOSTIC BRANDED|NATIONAL|Spend>,
 <gurobi.Var DTC SEARCH-RA PSA AGNOSTIC BRANDED|NATIONAL|Revenue>,
 <gurobi.Var DTC SEARCH-UC BRANDED|NATIONAL|Spend>,
 <gurobi.Var DTC SEARCH-UC BRANDED|NATIONAL|Revenue>,
 <gurobi.Var DTC SOCIAL-RA PSA BRANDED|NATIONAL|Spend>,
 <gurobi.Var DTC SOCIAL-RA PSA BRANDED|NATIONAL|Revenue>,
 <gurobi.Var DTC SOCIAL-UC BRANDED|NATIONAL|Spend>,
 <gurobi.Var DTC SOCIAL-UC BRANDED|NATIONAL|Revenue>,
 <gurobi.Var DTC TV-PSA BRANDED|NATIONAL|Spend>,
 <gurobi.Var DTC TV-PSA BRANDED|NATIONAL|Revenue>,
 <gurobi.Var DTC TV-RA BRANDED|NATIONAL|Spend>,
 <gurobi.Var DTC TV-RA BRANDED|NATIONAL|Revenue>,
 <gurobi.Var DTC

In [76]:
# Tag the variables and constraints

# Define tags for some variables in order to access their values later
for count, v in enumerate(model.getVars()):
    v.VTag = "Variable{}".format(v)

In [77]:
batchID = model.optimizeBatch()

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)


In [73]:
batchID

'2e0e887e-5822-4e3d-91af-11ceeb5046b4'

In [ ]:
    try:
        with gp.Env(empty=True) as env:

            #Compute Server:
            #env.setParam("CompteServer", "myserver1:32123")
            #env.setParam("ServerPassword", "pass")
            
            #Cluser Manager:
            env.setParam("CSManager","10.192.71.173:61080")
            env.setParam("CSAPIAccessID", "5bcf91fe-dbd5-4b16-bb1e-676001613076")
            env.setParam("CSAPISecret", "ad261c20-395a-4a86-b059-bd6d7779d132")

            # Instant Cloud:
            #env.setParam("CloudHost","cloud.gurobi.com")
            #env.setParam("CloudAccessID","78bf9c04-d019-4a7d-ba88-58283915823a")
            #env.setParam("CloudSecretKey", "YzVhMmU0YjgtYzVmNS00Nz")
            #env.setParam("CloudPool","498085-default")
            #env.setParam("GURO_PAR_CROSSOVERPI",0)
            env.start()

            with gp.Model("promomix-model", env=env) as model:

                # Define the variable for optimization
                # Can define bounds while declaration of varaibles  - both Upper/Lower Bounds
                x = {}
                y = {}

                for i in range(num_channels):
                    x[i] = model.addVar(vtype = GRB.CONTINUOUS,lb = float(inp_df[inp_df[['channel', 'segment']].agg(tuple, 1) == channels_to_opt[i]]['spend_lower_bound']),ub = float(inp_df[inp_df[['channel', 'segment']].agg(tuple, 1) == channels_to_opt[i]]['spend_upper_bound']), name = ('|'.join(channels_to_opt[i]) + '|' + 'Spend'))
                    y[i] = model.addVar(vtype = GRB.CONTINUOUS, lb = (-gp.GRB.INFINITY), ub = gp.GRB.INFINITY, name = ('|'.join(channels_to_opt[i]) + '|' + 'Revenue'))

                # Constraint:
                if(''.join(inp_df['opt_max_criteria'].unique()).lower()!='recommend optimal spend'):
                    model.addConstr(gp.quicksum(x[i] for i in range(num_channels)) == total_spend, "Total Spend Constraint ")

                # Add piecewise-linear constraint - Response Curves

                if(''.join(inp_df['opt_max_func'].unique()).lower() == "sales"):
                    for i in range(num_channels):
                        model.addGenConstrPWL(x[i], y[i], inp_resp_df[inp_resp_df[['channel', 'segment']].agg(tuple, 1) == channels_to_opt[i]]['spend'].tolist(), inp_resp_df[inp_resp_df[['channel', 'segment']].agg(tuple, 1) == channels_to_opt[i]]['net_lt_incremental_revenue'].tolist(), "pwl")
                elif(''.join(inp_df['opt_max_func'].unique()).lower() == "profit"):
                     for i in range(num_channels):
                        model.addGenConstrPWL(x[i], y[i], inp_resp_df[inp_resp_df[['channel', 'segment']].agg(tuple, 1) == channels_to_opt[i]]['spend'].tolist(), inp_resp_df[inp_resp_df[['channel', 'segment']].agg(tuple, 1) == channels_to_opt[i]]['net_lt_profit'].tolist(), "pwl")

                # Objective Function

                model.setObjective(gp.quicksum(y[i] for i in range(num_channels)), GRB.MAXIMIZE)

                # Optimize
                model.optimize()

                status = model.Status

                if status in (GRB.INF_OR_UNBD, GRB.INFEASIBLE, GRB.UNBOUNDED):
                    print('Model cannot be solved because it is infeasible or unbounded')
                    sys.exit(0)

                if status != GRB.OPTIMAL:
                    print('Optimization was stopped with status ' + str(status))
                    sys.exit(0)

                for v in model.getVars():
                    if "spend" in v.Varname.lower():
                        opt_tuple = opt_tuple + [(v.Varname.split('|')[0].strip(),v.Varname.split('|')[1].strip(), v.X),]


In [ ]:
env.dispose()

### Optimization Section

In [6]:
def run_optimization_scn(inp_df,inp_resp_df):   
    
    ########################################################################
    ## 1. Pre - Processing on Input table based on constraints selections ##
    ########################################################################

    # Modify Input Data based on National/ Segment Selection
    chnl_natl_only = [x for x in inp_df[inp_df['level'].str.lower() == 'level 4 - subchannel']['channel'].unique() if x not in inp_df[inp_df['level'].str.lower() == 'segment']['channel'].unique()]

    if(''.join(inp_df['opt_channel_level'].unique()).lower() == 'national'):
        inp_df = inp_df[inp_df['level'].str.lower() == 'level 4 - subchannel']
    elif(''.join(inp_df['opt_channel_level'].unique()).lower() == 'segment'):
        inp_df = inp_df[(inp_df['level'].str.lower() == 'segment') | ((inp_df['level'].str.lower() == 'level 4 - subchannel') & (inp_df['channel'].isin(chnl_natl_only)))]

    # Identify channel - segment selected by user for optimization
    channels_to_opt = inp_df[inp_df['opt_chnl_selected_flg']==1][['channel', 'segment']].agg(tuple, 1).unique()

    # Sales/Profit Maximization Criteria (Remove the component of spends which are not to be optimized)

    if(''.join(inp_df['opt_max_criteria'].unique()).lower() =='only reallocate spend'):
        total_spend = inp_df['spend'].sum() - (inp_df[inp_df['opt_chnl_selected_flg']==0]['spend'].sum() if 0 in inp_df['opt_chnl_selected_flg'] else 0)
    elif(''.join(inp_df['opt_max_criteria'].unique()).lower() =='change spend to'):
        total_spend = float(inp_df['user_inp_change_spend_to'].unique()) - (inp_df[inp_df['opt_chnl_selected_flg']==0]['spend'].sum() if 0 in inp_df['opt_chnl_selected_flg'] else 0 )
    elif(''.join(inp_df['opt_max_criteria'].unique()).lower() =='recommend optimal spend'):
        total_spend = 100000*inp_df['spend'].sum() - (inp_df[inp_df['opt_chnl_selected_flg']==0]['spend'].sum() if 0 in inp_df['opt_chnl_selected_flg'] else 0 ) 

    #########################
    ## 2. Run Optimization ##
    #########################

    optimized_results = optimize_func(inp_df,inp_resp_df,channels_to_opt)

    # Join the optimized spend with input data
    opt_df = inp_df.merge(pd.DataFrame(optimized_results, columns=['channel', 'segment', 'sim_spend']),how='left', left_on=['channel','segment'], right_on = ['channel','segment'])
    opt_df['sim_spend'] = opt_df['sim_spend'].fillna(opt_df['spend'])
    
    #######################################################################
    ## 3. Post Optimization Calculation of metrics (sales, profit, etc.) ##
    #######################################################################

    # Please fill the formula based on simulation table calculations
    
    # calc revenue
#     opt_df['sim_gross_lt_incremental_revenue'] = #opt_df['beta'] * math.log(1+opt_df['k_value']*(opt_df['sim_spend']/(opt_df['spend']*opt_df['activity'])/opt_df['reach']/opt_df['modelling_months']) - opt_df['c_value'])
#     opt_df['sim_net_lt_incremental_revenue']

#     # calc profit
#     opt_df['sim_gross_lt_profit']
#     opt_df['sim_net_lt_profit']

#     # calc roi
#     opt_df['sim_gross_lt_roi']
#     opt_df['sim_net_lt_roi']

#     # calc mroi
#     opt_df['sim_gross_lt_mroi']
#     opt_df['sim_net_lt_mroi']

#     update the rds

    return(opt_df)

#### Code starts here

In [7]:
def optimize_func(inp_df,inp_resp_df,channels_to_opt):
    opt_tuple = []
    num_channels = len(channels_to_opt)
    try:
        with gp.Env(empty=True) as env:

            #Compute Server:
            #env.setParam("CompteServer", "myserver1:32123")
            #env.setParam("ServerPassword", "pass")
            
            #Cluser Manager:
            env.setParam("CSManager","10.192.71.173:61080")
            env.setParam("CSAPIAccessID", "5bcf91fe-dbd5-4b16-bb1e-676001613076")
            env.setParam("CSAPISecret", "ad261c20-395a-4a86-b059-bd6d7779d132")

            # Instant Cloud:
            #env.setParam("CloudHost","cloud.gurobi.com")
            #env.setParam("CloudAccessID","78bf9c04-d019-4a7d-ba88-58283915823a")
            #env.setParam("CloudSecretKey", "YzVhMmU0YjgtYzVmNS00Nz")
            #env.setParam("CloudPool","498085-default")
            #env.setParam("GURO_PAR_CROSSOVERPI",0)
            env.start()

            with gp.Model("promomix-model", env=env) as model:

                # Define the variable for optimization
                # Can define bounds while declaration of varaibles  - both Upper/Lower Bounds
                x = {}
                y = {}

                for i in range(num_channels):
                    x[i] = model.addVar(vtype = GRB.CONTINUOUS,lb = float(inp_df[inp_df[['channel', 'segment']].agg(tuple, 1) == channels_to_opt[i]]['spend_lower_bound']),ub = float(inp_df[inp_df[['channel', 'segment']].agg(tuple, 1) == channels_to_opt[i]]['spend_upper_bound']), name = ('|'.join(channels_to_opt[i]) + '|' + 'Spend'))
                    y[i] = model.addVar(vtype = GRB.CONTINUOUS, lb = (-gp.GRB.INFINITY), ub = gp.GRB.INFINITY, name = ('|'.join(channels_to_opt[i]) + '|' + 'Revenue'))

                # Constraint:
                if(''.join(inp_df['opt_max_criteria'].unique()).lower()!='recommend optimal spend'):
                    model.addConstr(gp.quicksum(x[i] for i in range(num_channels)) == total_spend, "Total Spend Constraint ")

                # Add piecewise-linear constraint - Response Curves

                if(''.join(inp_df['opt_max_func'].unique()).lower() == "sales"):
                    for i in range(num_channels):
                        model.addGenConstrPWL(x[i], y[i], inp_resp_df[inp_resp_df[['channel', 'segment']].agg(tuple, 1) == channels_to_opt[i]]['spend'].tolist(), inp_resp_df[inp_resp_df[['channel', 'segment']].agg(tuple, 1) == channels_to_opt[i]]['net_lt_incremental_revenue'].tolist(), "pwl")
                elif(''.join(inp_df['opt_max_func'].unique()).lower() == "profit"):
                     for i in range(num_channels):
                        model.addGenConstrPWL(x[i], y[i], inp_resp_df[inp_resp_df[['channel', 'segment']].agg(tuple, 1) == channels_to_opt[i]]['spend'].tolist(), inp_resp_df[inp_resp_df[['channel', 'segment']].agg(tuple, 1) == channels_to_opt[i]]['net_lt_profit'].tolist(), "pwl")

                # Objective Function

                model.setObjective(gp.quicksum(y[i] for i in range(num_channels)), GRB.MAXIMIZE)

                # Optimize
                model.optimize()

                status = model.Status

                if status in (GRB.INF_OR_UNBD, GRB.INFEASIBLE, GRB.UNBOUNDED):
                    print('Model cannot be solved because it is infeasible or unbounded')
                    sys.exit(0)

                if status != GRB.OPTIMAL:
                    print('Optimization was stopped with status ' + str(status))
                    sys.exit(0)

                for v in model.getVars():
                    if "spend" in v.Varname.lower():
                        opt_tuple = opt_tuple + [(v.Varname.split('|')[0].strip(),v.Varname.split('|')[1].strip(), v.X),]

    except gurobipy.GurobiError as e:
        print('Error code ' + str(e.errno) + ": " + str(e))
    except AttributeError as e:
        print('Encountered an attribute error: ' + str(e))
    finally:
        # Safely release memory and/or server side resources consumed by
        # the default environment.
        env.dispose()
        return(opt_tuple)

### Calls optimization at the end

In [8]:
inter_table_df = run_optimization_scn(inp_df = promomix_df,inp_resp_df = response_curve_df)

Changed value of parameter CSManager to 10.192.71.173:61080
   Prev:   Default: 
Changed value of parameter CSAPIAccessID to 5bcf91fe-dbd5-4b16-bb1e-676001613076
   Prev:   Default: 
Changed value of parameter CSAPISecret
Using license file C:\gurobi903\gurobi.lic
Set parameter CSManager to value 10.192.71.173:61080
Set parameter CSAPIAccessID to value 5bcf91fe-dbd5-4b16-bb1e-676001613076
Set parameter CSAPISecret
Compute Server job ID: 69d4c6e0-371a-4e16-80d8-ac2e0ce7a1e0
Capacity available on '10.192.71.215' - connecting...
Established HTTP unencrypted connection

Compute Server communication statistics:
  Sent: 0.001 MBytes in 7 msgs and 4.07s (0.00 MB/s)
  Received: 0.001 MBytes in 4 msgs and 2.36s (0.00 MB/s)



In [65]:
inter_table_df = run_optimization_scn(inp_df = promomix_df,inp_resp_df = response_curve_df)

Changed value of parameter CSManager to 10.192.71.173:61080
   Prev:   Default: 
Changed value of parameter CSAPIAccessID to 5bcf91fe-dbd5-4b16-bb1e-676001613076
   Prev:   Default: 
Changed value of parameter CSAPISecret
Using license file C:\gurobi903\gurobi.lic
Set parameter CSManager to value 10.192.71.173:61080
Set parameter CSAPIAccessID to value 5bcf91fe-dbd5-4b16-bb1e-676001613076
Set parameter CSAPISecret
Compute Server job ID: 733e4043-c992-49d4-9324-728073a87c57
Capacity available on '10.192.71.215:61080' - connecting...
Established HTTP unencrypted connection
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 1 rows, 32 columns and 16 nonzeros
Model fingerprint: 0x332c0dae
Model has 16 general constraints
Variable types: 32 continuous, 0 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+06, 9e+07]
  RHS range        [3e+08, 3e+08]
Presolve added 32 rows and 640 colu

In [69]:
inter_table_df.head()

,country,brand,segment,period,level,segment_level,channel_category,channel_type,channel,sub_channel,...,frequency,gtn,net_sales,spend_lower_bound,spend_upper_bound,opt_max_func,opt_max_criteria,opt_channel_level,user_inp_change_spend_to,sim_spend
0,USA,XELJANZ,NATIONAL,FY2019,LEVEL 4 - Subchannel,NATIONAL,MCM,DIRECTMAIL,DIRECTMAIL,DIRECTMAIL,...,0.179982,0.5,9.209038e+08,1.057000e+05,1.585500e+05,Sales,Only reallocate spend,National,NaN,1.321250e+05
1,USA,XELJANZ,NATIONAL,FY2019,LEVEL 4 - Subchannel,NATIONAL,DTC,DTC DISPLAY/SOCIAL,DTC DISPLAY-RA PSA BRANDED,RA PSA BRANDED,...,417690.549442,0.5,1.114554e+09,8.235574e+06,1.235336e+07,Sales,Only reallocate spend,National,NaN,1.235336e+07
2,USA,XELJANZ,NATIONAL,FY2019,LEVEL 4 - Subchannel,NATIONAL,DTC,DTC DISPLAY/SOCIAL,DTC DISPLAY-UC BRANDED,UC BRANDED,...,398987.853668,0.5,1.114554e+09,3.916996e+06,5.875495e+06,Sales,Only reallocate spend,National,NaN,5.875495e+06
3,USA,XELJANZ,NATIONAL,FY2019,LEVEL 4 - Subchannel,NATIONAL,DTC,DTC PRINT,DTC PRINT-RA BRANDED,RA BRANDED,...,0.067375,0.5,1.114554e+09,3.010110e+06,4.515166e+06,Sales,Only reallocate spend,National,NaN,4.515166e+06
4,USA,XELJANZ,NATIONAL,FY2019,LEVEL 4 - Subchannel,NATIONAL,DTC,DTC SEARCH,DTC SEARCH-RA PSA AGNOSTIC BRANDED,RA PSA AGNOSTIC BRANDED,...,2552.465311,0.5,1.114554e+09,1.492905e+07,2.239358e+07,Sales,Only reallocate spend,National,NaN,1.753866e+07


In [9]:
inter_table_df.head()

,country,brand,segment,period,level,segment_level,channel_category,channel_type,channel,sub_channel,...,frequency,gtn,net_sales,spend_lower_bound,spend_upper_bound,opt_max_func,opt_max_criteria,opt_channel_level,user_inp_change_spend_to,sim_spend
0,USA,XELJANZ,NATIONAL,FY2019,LEVEL 4 - Subchannel,NATIONAL,MCM,DIRECTMAIL,DIRECTMAIL,DIRECTMAIL,...,0.179982,0.5,9.209038e+08,1.057000e+05,1.585500e+05,Sales,Only reallocate spend,National,NaN,132125.00
1,USA,XELJANZ,NATIONAL,FY2019,LEVEL 4 - Subchannel,NATIONAL,DTC,DTC DISPLAY/SOCIAL,DTC DISPLAY-RA PSA BRANDED,RA PSA BRANDED,...,417690.549442,0.5,1.114554e+09,8.235574e+06,1.235336e+07,Sales,Only reallocate spend,National,NaN,10294467.21
2,USA,XELJANZ,NATIONAL,FY2019,LEVEL 4 - Subchannel,NATIONAL,DTC,DTC DISPLAY/SOCIAL,DTC DISPLAY-UC BRANDED,UC BRANDED,...,398987.853668,0.5,1.114554e+09,3.916996e+06,5.875495e+06,Sales,Only reallocate spend,National,NaN,4896245.43
3,USA,XELJANZ,NATIONAL,FY2019,LEVEL 4 - Subchannel,NATIONAL,DTC,DTC PRINT,DTC PRINT-RA BRANDED,RA BRANDED,...,0.067375,0.5,1.114554e+09,3.010110e+06,4.515166e+06,Sales,Only reallocate spend,National,NaN,3762638.01
4,USA,XELJANZ,NATIONAL,FY2019,LEVEL 4 - Subchannel,NATIONAL,DTC,DTC SEARCH,DTC SEARCH-RA PSA AGNOSTIC BRANDED,RA PSA AGNOSTIC BRANDED,...,2552.465311,0.5,1.114554e+09,1.492905e+07,2.239358e+07,Sales,Only reallocate spend,National,NaN,18661313.95
